In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn import metrics
from sklearn.model_selection import train_test_split

from google.colab import drive

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os   # This module is used for interacting with the operating system. It provides a way to work with files and directories.

import tensorflow as tf
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# --- Load data and setup directories ---
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/TCGA-BRCA_1079.zip

#'r' before the string indicates a raw string ('\' are treated as literal and not escape characters)
result_path = r"results"

if not os.path.exists(result_path):
    os.makedirs(result_path)

Mounted at /content/gdrive
Archive:  gdrive/MyDrive/TCGA-BRCA_1079.zip
  inflating: TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv  


In [ ]:
class AE:
    def __init__(self,data,dims):
        self.data = data
        #p_split = 0.9
        #idx_split = int(p_split * data.shape[0])
        #self.x, self.x_val = np.split(self.data, [idx_split])
        #print("Training data shape: ", self.x.shape)
        #print("Validation data shape: ", self.x_val.shape)
        self.x, self.x_val = train_test_split(self.data, test_size=0.1, train_size=0.9)


        self.input_dim = data.shape[1]
        self.encoding1_dim = dims[0]
        self.encoding2_dim = dims[1]
        self.encoding3_dim = dims[2]
        self.middle_dim = dims[3]

        # this is our input placeholder
        self.input_factor = Input(shape=(self.input_dim,))
        self.encoded = Dense(self.encoding1_dim, activation='sigmoid')(self.input_factor)
        self.encoded = Dense(self.encoding2_dim, activation='sigmoid')(self.encoded)
        self.encoded = Dense(self.encoding3_dim, activation='sigmoid')(self.encoded)

        self.encoder_output = Dense(self.middle_dim, activation='sigmoid')(self.encoded)

        self.decoded = Dense(self.encoding3_dim, activation='sigmoid')(self.encoder_output)
        self.decoded = Dense(self.encoding2_dim, activation='sigmoid')(self.decoded)
        self.decoded = Dense(self.encoding1_dim, activation='sigmoid')(self.decoded)
        self.decoded = Dense(self.input_dim)(self.decoded)

        self.autoencoder = Model(inputs=self.input_factor, outputs=self.decoded)

        self.encoder = Model(inputs=self.input_factor, outputs=self.encoder_output)

        # compile autoencoder
        # the initial learning rate for the 'adam' optimizer is 0.001
        custom_adam_optimizer = Adam(learning_rate=0.0005)
        self.autoencoder.compile(optimizer=custom_adam_optimizer, loss='mse')

    def save(self):
        self.autoencoder.save("./trained_model/model.h5")

    def predict(self,data):
        return self.encoder.predict(data)

In [ ]:
class DAE(AE):
    def __init__(self,data,dims,noise_factor):
        AE.__init__(self,data,dims)
        self.noise_factor=noise_factor
        self.x=self.x + noise_factor * np.random.normal(0.0, 1.0, self.x.shape)
        self.x_val=self.x_val + noise_factor * np.random.normal(0.0, 1.0, self.x_val.shape)


    def train(self):
        callback_metrics = WandbMetricsLogger()
        early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=50, min_delta=0.01, restore_best_weights=True)

        self.autoencoder.fit(self.x, self.x, validation_data=(self.x_val, self.x_val), epochs=500, verbose=1, batch_size=32, shuffle=True, callbacks=[callback_metrics, early_stopping])

In [ ]:
# --- Prepare input data ---
input_data_frame = pd.read_csv("TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv", sep="\t")   # read .tsv file into memory
input_data = input_data_frame.values  # retrieve values as numpy array
print ("Shape of input data: ", input_data.shape) # rows are genes and columns are samples
print(input_data)

input_data = np.transpose(input_data) # rows are samples and columns are genes
input_data = np.delete(input_data, (0), axis=0) # delete first row of the data matrix
print ("Shape of prepared input data: ", input_data.shape)
print(input_data)

input_data = np.asarray(input_data).astype("float32") # Every character/number encoded with 32 Bit
print("--- Converted to Float32 ---")
print(input_data)

Shape of input data:  (17162, 1080)
[['A1BG' 0.1719444908272692 0.7300083449184781 ... 0.614468050540426
  1.3458846686508648 0.7186659277414379]
 ['A1CF' -0.814924951368837 2.386837959210663 ... 3.016626468680079
  -0.814924951368837 0.2356343444931301]
 ['A2M' -0.946474342215696 -0.5321448682087864 ... 0.5601782905367024
  1.512226893819039 -0.6446442624050204]
 ...
 ['DHCR7' 0.9104445032812696 0.3842357893348481 ... 0.7718469491250562
  -0.4217551872682497 -0.970509103357415]
 ['TMEM45B' -1.9635765960806173 1.268061022702539 ... -0.3477150909879061
  0.4043269887748967 -0.5444141861092432]
 ['GPR160' 0.6427457224400778 1.759067645972089 ... 0.3169998035848742
  -0.7956005603903231 -0.1338926768376846]]
Shape of prepared input data:  (1079, 17162)
[[0.1719444908272692 -0.814924951368837 -0.946474342215696 ...
  0.9104445032812696 -1.9635765960806173 0.6427457224400778]
 [0.7300083449184781 2.386837959210663 -0.5321448682087864 ...
  0.3842357893348481 1.268061022702539 1.759067645972

In [ ]:
encoding1_dim = 8000
encoding2_dim = 1000
encoding3_dim = 200
middle_dim = 10
dims = [encoding1_dim, encoding2_dim, encoding3_dim, middle_dim]
noise_factor = 0.01


for i in range(1, 11):
    # initialize a new run (<=> single unit of computation)
    run = wandb.init(project="DAE_fitting",
                     # Set the project where this run will be logged
                     name=f"random_data_split",  # We pass a run name otherwise it’ll be randomly assigned
                     # Track hyperparameters and run metadata
                     config={
                         "optimizer": "adam",
                         "validation split": 0.1,
                         "loss": "mse",
                         "training_set": "BRCA",
                         "test_set": "none",
                         "batch_size": 32,
                         "encoding_layers": 3,
                         "activation_functions": "sigmoid, sigmoid, sigmoid, identity",
                         "noise_factor": 0.01
                     })

    dae = DAE(input_data, dims, noise_factor)
    dae.autoencoder.summary()
    dae.train()

    # Save embedded encodings
    encoded_factors = dae.predict(input_data)
    if not os.path.exists("{p}/DAE_EM_{i}.txt".format(p=result_path, i=i)):
        os.mknod("{p}/DAE_EM_{i}.txt".format(p=result_path, i=i))
    np.savetxt("{p}/DAE_EM_{i}.txt".format(p=result_path, i=i), encoded_factors)

    # Mark the run as finished
    wandb.finish()

wandb: Currently logged in as: cosybio-compsysmed. Use `wandb login --relogin` to force relogin


Training data shape:  (971, 17162)
Validation data shape:  (108, 17162)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 17162)]           0         
                                                                 
 dense (Dense)               (None, 8000)              137304000 
                                                                 
 dense_1 (Dense)             (None, 1000)              8001000   
                                                                 
 dense_2 (Dense)             (None, 200)               200200    
                                                                 
 dense_3 (Dense)             (None, 10)                2010      
                                                                 
 dense_4 (Dense)             (None, 200)               2200      
                                                       

31/31 [==============================] - 7s 79ms/step - loss: 1.1041 - val_loss: 1.1071
Epoch 2/500
31/31 [==============================] - 1s 19ms/step - loss: 0.9882 - val_loss: 1.1049
Epoch 3/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9878 - val_loss: 1.1077
Epoch 4/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9875 - val_loss: 1.1100
Epoch 5/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9872 - val_loss: 1.1131
Epoch 6/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9870 - val_loss: 1.1168
Epoch 7/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9867 - val_loss: 1.1205
Epoch 8/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9862 - val_loss: 1.1247
Epoch 9/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9854 - val_loss: 1.1317
Epoch 10/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9839 - val_loss: 1.1375
Epoch 11/50

epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,████▇▇▆▆▆▆▆▆▆▅▅▅▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,▇███▇▇▆▆▆▆▆▆▆▅▅▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,169
epoch/learning_rate,0.0005
epoch/loss,0.7106
epoch/val_loss,0.76062


Training data shape:  (971, 17162)
Validation data shape:  (108, 17162)
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 17162)]           0         
                                                                 
 dense_8 (Dense)             (None, 8000)              137304000 
                                                                 
 dense_9 (Dense)             (None, 1000)              8001000   
                                                                 
 dense_10 (Dense)            (None, 200)               200200    
                                                                 
 dense_11 (Dense)            (None, 10)                2010      
                                                                 
 dense_12 (Dense)            (None, 200)               2200      
                                                     

...

31/31 [==============================] - 4s 68ms/step - loss: 1.1032 - val_loss: 1.1037
Epoch 2/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9884 - val_loss: 1.1028
Epoch 3/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9879 - val_loss: 1.1057
Epoch 4/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9876 - val_loss: 1.1087
Epoch 5/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9873 - val_loss: 1.1121
Epoch 6/500
31/31 [==============================] - 1s 17ms/step - loss: 0.9871 - val_loss: 1.1157
Epoch 7/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9869 - val_loss: 1.1192
Epoch 8/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9868 - val_loss: 1.1224
Epoch 9/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9865 - val_loss: 1.1264
Epoch 10/500
31/31 [==============================] - 1s 17ms/step - loss: 0.9860 - val_loss: 1.1289
Epoch 11/50

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,████▇▇▆▆▆▆▆▅▅▅▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_loss,▇███▇▇▇▆▆▅▅▅▅▄▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,187
epoch/learning_rate,0.0005
epoch/loss,0.70232
epoch/val_loss,0.76488


Training data shape:  (971, 17162)
Validation data shape:  (108, 17162)
Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 17162)]           0         
                                                                 
 dense_72 (Dense)            (None, 8000)              137304000 
                                                                 
 dense_73 (Dense)            (None, 1000)              8001000   
                                                                 
 dense_74 (Dense)            (None, 200)               200200    
                                                                 
 dense_75 (Dense)            (None, 10)                2010      
                                                                 
 dense_76 (Dense)            (None, 200)               2200      
                                                    

31/31 [==============================] - 4s 66ms/step - loss: 1.1106 - val_loss: 1.1092
Epoch 2/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9882 - val_loss: 1.1065
Epoch 3/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9877 - val_loss: 1.1084
Epoch 4/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9875 - val_loss: 1.1111
Epoch 5/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9872 - val_loss: 1.1138
Epoch 6/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9871 - val_loss: 1.1175
Epoch 7/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9869 - val_loss: 1.1204
Epoch 8/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9866 - val_loss: 1.1211
Epoch 9/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9861 - val_loss: 1.1270
Epoch 10/500
31/31 [==============================] - 1s 18ms/step - loss: 0.9855 - val_loss: 1.1306
Epoch 11/50

epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,████▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁
epoch/val_loss,▇███▇▇▆▆▆▆▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,173
epoch/learning_rate,0.0005
epoch/loss,0.7261
epoch/val_loss,0.76763
